In [1]:
# # Tensor Boards
# %load_ext tensorboard
# # %reload_ext tensorboard
# %tensorboard --logdir=/Users/emilytang/Library/CloudStorage/GoogleDrive-etangs@g.ucla.edu/My\ Drive/UCLA\ Winter\ 2025/ECE\ C247/C247-Final-Project/logs
# # %tensorboard --logdir="H:\My Drive\UCLA Winter 2025\ECE C247\C247-Final-Project\logs"

In [83]:
import os
import plotly.graph_objects as go
from tensorboard.backend.event_processing import event_accumulator
import yaml
import re
import numpy as np

# root_dir = r"H:\My Drive\UCLA Winter 2025\ECE C247 Final Project\final-logs"
root_dir = r'/Users/emilytang/Library/CloudStorage/GoogleDrive-etangs@g.ucla.edu/My Drive/UCLA Winter 2025/ECE C247 Final Project/final-logs'

event_files = []
# Walk through the directory tree
for root, dirs, files in os.walk(root_dir):
    # Check if the directory contains 'lightning_logs'
    if 'lightning_logs' in dirs:
        lightning_log_dir = os.path.join(root, 'lightning_logs')

        # Walk through the 'lightning_logs' directory and get all event files
        for sub_root, sub_dirs, sub_files in os.walk(lightning_log_dir):
            for file in sub_files:
                # Filter event files (e.g., those that start with 'events.out.tfevents')
                if file.startswith('events.out.tfevents'):
                    event_files.append(os.path.join(sub_root, file))
event_files.sort()

import matplotlib.pyplot as plt
from pylab import *
colors = [matplotlib.colors.rgb2hex(c) for c in plt.cm.tab20(np.linspace(0,1,len(event_files)))]
fig = go.Figure()

yaml_files=[]
for c, log_dir in enumerate(event_files):
    rel_path = os.path.relpath(log_dir, root_dir)
    test_name = rel_path[:rel_path.find('/')]
    event_acc = event_accumulator.EventAccumulator(log_dir)
    event_acc.Reload()
    parent_directory = os.path.abspath(os.path.join(log_dir, '..', '..', '..', 'emg2qwerty.log'))

    with open(parent_directory, 'r') as file:
        content = file.read()
        yaml_content = "\n".join(re.findall(r"(?<=Config:)(.*?)(?=\[|\Z)", content, flags=re.DOTALL))
        log_dict = yaml.safe_load(yaml_content)
        train_transforms = '\n'.join([re.sub(r'\${(.*?)}', r'\1', item) for item in log_dict['transforms']['train']])
        batch_size = log_dict['batch_size']

    tags = event_acc.Tags()
    scalar_tags = tags.get("scalars", [])
    # try:
    if 'train/loss' in scalar_tags and 'val/CER' in scalar_tags:
        # Extract scalar data (replace 'loss' and 'accuracy' with the correct tag names if needed)
        losses = event_acc.Scalars('train/loss')  # Replace with your tag for loss
        val_CERs = event_acc.Scalars('val/CER')  # Replace with your tag for accuracy

        # Extract the x (step) and y (value) for plotting
        train_step = [event_acc.Scalars('epoch')[-1].value*e.step/losses[-1].step for e in losses]
        train_loss = [e.value for e in losses]

        epochs = event_acc.Scalars('epoch')
        val_CER = [e.value for e in val_CERs]
        epoch = [e.value for e in epochs]

        val_per_epoch = [val_CER[int(i)-1] for i in epoch]
        
        # Plot the loss curve
        fig.add_trace(
            go.Scatter(
                x=train_step,
                y=train_loss,
                mode='lines+markers',
                name=f'{test_name}',
                line=dict(color=colors[c]),
                hovertemplate=(
                    f'<b>{test_name}</b><br>'
                    'Epoch: %{x:i}<br>'
                    'train/loss: %{y}<br>'
                )
            )
        )
        yaml_files.append(yaml_content)
# Create layout with log scale for both y-axes
fig.update_layout(
    width=900,
    height=500,
    title="Training Loss",
    xaxis_title="Steps",
    yaxis=dict(
        title="Loss",
        type="log",
        exponentformat='power',
        title_font=dict(size=14),
        tickfont=dict(size=12),
        dtick=1
    ),
    legend_title="Model",
    template="seaborn",
    showlegend=True
)

fig.show()


In [ ]:
import os
import plotly.graph_objects as go
from tensorboard.backend.event_processing import event_accumulator
import yaml
import re
import numpy as np

# root_dir = r"H:\My Drive\UCLA Winter 2025\ECE C247 Final Project\final-logs"
root_dir = r'/Users/emilytang/Library/CloudStorage/GoogleDrive-etangs@g.ucla.edu/My Drive/UCLA Winter 2025/ECE C247 Final Project/final-logs'

event_files = []
# Walk through the directory tree
for root, dirs, files in os.walk(root_dir):
    # Check if the directory contains 'lightning_logs'
    if 'lightning_logs' in dirs:
        lightning_log_dir = os.path.join(root, 'lightning_logs')

        # Walk through the 'lightning_logs' directory and get all event files
        for sub_root, sub_dirs, sub_files in os.walk(lightning_log_dir):
            for file in sub_files:
                # Filter event files (e.g., those that start with 'events.out.tfevents')
                if file.startswith('events.out.tfevents'):
                    event_files.append(os.path.join(sub_root, file))
event_files.sort()

import matplotlib.pyplot as plt
from pylab import *
colors = [matplotlib.colors.rgb2hex(c) for c in plt.cm.tab20(np.linspace(0,1,len(event_files)))]
fig = go.Figure()

yaml_files=[]
for c, log_dir in enumerate(event_files):
    rel_path = os.path.relpath(log_dir, root_dir)
    test_name = rel_path[:rel_path.find('/')]
    event_acc = event_accumulator.EventAccumulator(log_dir)
    event_acc.Reload()
    parent_directory = os.path.abspath(os.path.join(log_dir, '..', '..', '..', 'emg2qwerty.log'))

    with open(parent_directory, 'r') as file:
        content = file.read()
        yaml_content = "\n".join(re.findall(r"(?<=Config:)(.*?)(?=\[|\Z)", content, flags=re.DOTALL))
        log_dict = yaml.safe_load(yaml_content)
        train_transforms = '\n'.join([re.sub(r'\${(.*?)}', r'\1', item) for item in log_dict['transforms']['train']])
        batch_size = log_dict['batch_size']

    tags = event_acc.Tags()
    scalar_tags = tags.get("scalars", [])
    # try:
    if 'train/loss' in scalar_tags and 'val/CER' in scalar_tags:
        # Extract scalar data (replace 'loss' and 'accuracy' with the correct tag names if needed)
        losses = event_acc.Scalars('train/loss')  # Replace with your tag for loss
        val_CERs = event_acc.Scalars('val/CER')  # Replace with your tag for accuracy

        # Extract the x (step) and y (value) for plotting
        epochs = event_acc.Scalars('epoch')

        val_step = [event_acc.Scalars('epoch')[-1].value*e.step/losses[-1].step for e in val_CERs]
        val_CER = [e.value for e in val_CERs]
        epoch = [e.value for e in epochs]
        other_errors = []
        
        IERs = [e.value for e in event_acc.Scalars('val/IER')]
        DERs = [e.value for e in event_acc.Scalars('val/DER')]
        SERs = [e.value for e in event_acc.Scalars('val/SER')]
        


        # Plot the loss curve
        fig.add_trace(
            go.Scatter(
                x=val_step,
                y=val_CER,
                mode='lines+markers',
                name=f'{test_name}',
                line=dict(color=colors[c]),
                customdata=list(zip(IERs, DERs, SERs)),
                hovertemplate=(
                    f'<b>{test_name}</b><br>'
                    'Epoch/step: %{x}<br>'
                    'val/CER: %{y}<br>'
                    'val/IER: %{customdata[0]}<br>'
                    'val/DER: %{customdata[1]}<br>'
                    'val/SER: %{customdata[2]}<br>'
                    '<extra></extra>'
                )
            )
        )
        yaml_files.append(yaml_content)
# Create layout with log scale for both y-axes
fig.update_layout(
    width=900,
    height=500,
    title="val/CER by model",
    xaxis_title="Epoch",
    yaxis=dict(
        title="val/CER",
        type="log",
        exponentformat='power',
        title_font=dict(size=14),
        tickfont=dict(size=12),
        dtick=1,
    ),
    legend_title="Model",
    template="seaborn",
    showlegend=True
)

fig.show()
# fig.write_html("val_CER_plot.html")

In [46]:
scalar_tags

['hp_metric',
 'test/loss',
 'test/CER',
 'test/IER',
 'test/DER',
 'test/SER',
 'epoch']

Check files w Data

In [221]:
import os
import plotly.graph_objects as go
from tensorboard.backend.event_processing import event_accumulator
import yaml
import re
import numpy as np

# root_dir = r"/Users/emilytang/Library/CloudStorage/GoogleDrive-etangs@g.ucla.edu/My Drive/UCLA Winter 2025/ECE C247/C247-Final-Project/logs"
root_dir = r"/Users/emilytang/Documents/GitHub/C247-Final-Project/logs"
# root_dir = r'/Users/emilytang/Library/CloudStorage/GoogleDrive-etangs@g.ucla.edu/My Drive/UCLA Winter 2025/ECE C247 Final Project/final-logs'

event_files = []
# Walk through the directory tree
for root, dirs, files in os.walk(root_dir):
    # Check if the directory contains 'lightning_logs'
    if 'lightning_logs' in dirs:
        lightning_log_dir = os.path.join(root, 'lightning_logs')

        # Walk through the 'lightning_logs' directory and get all event files
        for sub_root, sub_dirs, sub_files in os.walk(lightning_log_dir):
            for file in sub_files:
                # Filter event files (e.g., those that start with 'events.out.tfevents')
                if file.startswith('events.out.tfevents'):
                    event_files.append(os.path.join(sub_root, file))
event_files.sort()

for c, log_dir in enumerate(event_files):
    rel_path = os.path.relpath(log_dir, root_dir)
    test_name = rel_path[:rel_path.find('/')]
    event_acc = event_accumulator.EventAccumulator(log_dir)
    event_acc.Reload()
    parent_directory = os.path.abspath(os.path.join(log_dir, '..', '..', '..', 'emg2qwerty.log'))

    with open(parent_directory, 'r') as file:
        content = file.read()
        yaml_content = "\n".join(re.findall(r"(?<=Config:)(.*?)(?=\[|\Z)", content, flags=re.DOTALL))
        log_dict = yaml.safe_load(yaml_content)
        train_transforms = '\n'.join([re.sub(r'\${(.*?)}', r'\1', item) for item in log_dict['transforms']['train']])
        batch_size = log_dict['batch_size']
    if batch_size == 32 and 'specaug' in train_transforms:
        print(f"TEST =============== {test_name}")
        print(train_transforms)
        print(f'BATCH SIZE={batch_size}')
        print(log_dir)


TEST =============== 2025-03-09-macbook
to_tensor
band_rotation
temporal_jitter
logspec
specaug
BATCH SIZE=32
/Users/emilytang/Documents/GitHub/C247-Final-Project/logs/2025-03-09-macbook/22-21-54/lightning_logs/version_0/events.out.tfevents.1741584115.MacBookPro.attlocal.net
TEST =============== 2025-03-09-macbook
to_tensor
band_rotation
temporal_jitter
logspec
specaug
BATCH SIZE=32
/Users/emilytang/Documents/GitHub/C247-Final-Project/logs/2025-03-09-macbook/22-21-54/lightning_logs/version_0/events.out.tfevents.1741595128.MacBookPro.attlocal.net
TEST =============== 2025-03-09-macbook
to_tensor
band_rotation
temporal_jitter
logspec
specaug
BATCH SIZE=32
/Users/emilytang/Documents/GitHub/C247-Final-Project/logs/2025-03-09-macbook/22-21-54/lightning_logs/version_0/events.out.tfevents.1741595156.MacBookPro.attlocal.net
